# Logistic Regression Assignment (due 26 November)

In this practical you will learn how to apply logistic regression to the task of predicting two digits from the MNIST database: http://yann.lecun.com/exdb/mnist/. The database contains 60000 train images containing digits and 10000 test images. The images are of size 28 × 28. We will use the images in a vectorized form: a vector of size of 784. The code extracting the digits 0 and 1 is provided in the stubs.


In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
print(tf.__version__)


2.12.0


In [4]:
import numpy as np 

In [5]:
def data_preprocess(images, labels):

    # number of examples m  
    m = images.shape[0]
    
    print(m)
    # create vector of ones to concatenate to our data matrix (for intercept terms)
    ones = np.ones(shape=[m, 1])
    images = np.concatenate((ones, images), axis=1)
    
    # to retrieve the images and corresponding labels where the label is either 0 or 1, 
    # we define two logical vectors that can be used to subset our data_matrices
    logical_mask_0 = labels == 0
    logical_mask_1 = labels == 1
    
    images_zeros = images[logical_mask_0]
    labels_zeros = labels[logical_mask_0]
    images_ones = images[logical_mask_1]
    labels_ones = labels[logical_mask_1]
    
    X = np.concatenate((images_zeros, images_ones), axis=0)
    y = np.concatenate((labels_zeros, labels_ones), axis=0)
    
    # shuffle the data and corresponding labels in unison
    def _shuffle_in_unison(a, b):
        assert len(a) == len(b)
        p = np.random.permutation(len(a))
        print('length ', len(a))
        print(a.shape)
        print(a[p].shape)
        return a[p], b[p]

    return _shuffle_in_unison(X,y)   

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0


11490434/11490434 [==============================] - 0s 0us/step


In [7]:
print (x_train.shape)

(60000, 28, 28)


In [8]:
x_train = x_train.reshape([60000,784])
x_test = x_test.reshape([10000,784])
print(x_train.shape)


(60000, 784)


In [9]:
X,y = data_preprocess(x_train, y_train)
print('shape: ', X.shape)
print('shape: ', y.shape)

60000
length  12665
(12665, 785)
(12665, 785)
shape:  (12665, 785)
shape:  (12665,)


Define hyperparams: learning rate and gradient descent steps


In [16]:
learning_rate = 0.01
gdc_steps = 1000

    

Initialize your parameters W


In [17]:
# number of features n
n = X.shape[1]
# we need to define our model parameters to be learned. we use W (weights) instead of theta this time.
mu, sigma = 0, 0.01 # mean and standard deviation
w = np.random.normal(mu, sigma, n)


In [18]:
print(X.shape, w.shape)

(12665, 785) (785,)


Define the sigmoid function, your code here:


In [19]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

Define the loss function as provided in equation 12 (Logistic regression slides)


In [20]:
def compute_cross_entropy_loss(y, y_hat):
    n = len(y)
    loss = 0
    for i in range(n):
        loss += -y[i] * np.log(y_hat[i]) - (1 - y[i]) * np.log(1 - y_hat[i])
    return loss / n



Start optimization. During training you minimize the loss function. In every iteration your loss should decrease. You also want to look how many correct predictions you have at every iteration. Reminder: the belonging to class digit 1 is when your prediction, $\hat y$ is greater or equal to 0.5. 

When you test your prediction vector (containing zero and ones) with the labels (also zero and ones) you can use the equal function. 

Example:
prediction = (1, 0, 1, 1) and the true labels are y = (0, 0, 1, 0).

When you test on equality you get following result: correct = (0, 1, 1, 0). Your accuracy is: 0+1+1+0
4 = 0.5.
You compute the accuracy for the training and test.

In [21]:
for step in range(0, gdc_steps):
    print("Performing step " + str(step) + " of gradient descent.")
    # perform the dot product between the weights and the examples
    z = np.dot(X,w)
    print('z', z, z.shape)
    # apply the nonlinearity
    y_hat = sigmoid(z)
    print("y hat: " + str(y_hat))
    # normally normalized with -1/m 
    loss = compute_cross_entropy_loss(y, y_hat)
    print("Loss at step " + str(step) + ": " + str(loss))
    
    # compute the error term, i.e. the difference between labels and estimated labels y_hat, see equation 24 in the slides
    error_term = y_hat - y
    
    # compute the gradient. as our data matrix X is currently layed out as X_j_i, we got to transpose it
    # see derived formula of the gradient calculation
    gradients = (1/len(y)) * np.dot(X.T, error_term)
    print(X.shape)
    print(error_term.shape)
    print(gradients.shape)
    
    # update w using the gdc update rule
    w = w-learning_rate*gradients
    
    # compute the predictions and cast them to int values
    predictions = np.int64(y_hat >= 0.5)
    print(predictions.shape)
    # compute mean accuracy
    accuracy = np.mean(predictions == y)
    print("Accuracy at step " + str(step) + ": " + str(accuracy))
    


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Performing step 500 of gradient descent.
z [ 4.41206362  3.12863191  2.02957809 ... -5.42279563  1.14367104
  2.07914355] (12665,)
y hat: [0.98801526 0.95805845 0.88386778 ... 0.00439538 0.75835301 0.88885945]
Loss at step 500: -5.3501933442619105
(12665, 785)
(12665,)
(785,)
(12665,)
Accuracy at step 500: 0.9971575207264114
Performing step 501 of gradient descent.
z [ 4.41465065  3.13029319  2.03090646 ... -5.42625219  1.14456346
  2.08055871] (12665,)
y hat: [0.98804585 0.95812516 0.88400406 ... 0.00438028 0.75851651 0.88899918]
Loss at step 501: -5.342353046421459
(12665, 785)
(12665,)
(785,)
(12665,)
Accuracy at step 501: 0.9971575207264114
Performing step 502 of gradient descent.
z [ 4.41723273  3.13195107  2.03223249 ... -5.4297027   1.14545445
  2.08197153] (12665,)
y hat: [0.98807631 0.95819162 0.88413996 ... 0.00436526 0.75867967 0.88913852]
Loss at step 502: -5.334541622757153
(12665, 785)
(12665,)
(785,)
(126

Evaluate model on test set

In [22]:
print("_______________________________")
print("Starting evaluation of test set")

X,y = data_preprocess(x_test, y_test)
z = np.dot(X,w) 
y_hat = sigmoid(z)
predictions = (y_hat>0.5).astype(np.int32)
accuracy = np.mean(predictions==y)
print("Accuracy of test set: " + str(accuracy))

_______________________________
Starting evaluation of test set
10000
length  2115
(2115, 785)
(2115, 785)
Accuracy of test set: 0.9990543735224586
